# 使用Embedding API
注：为了方便embedding api调用，应将密钥填入llm_universe下的.env文件，代码将自动读取并加载环境变量。
## 一、使用OpenAI API
GPT有封装好的接口，我们简单封装即可。目前GPT embedding mode有三种，性能如下所示：
|模型 | 每美元页数 | [MTEB](https://github.com/embeddings-benchmark/mteb)得分 | [MIRACL](https://github.com/project-miracl/miracl)得分|
| --- | --- | --- | --- |
|text-embedding-3-large|9,615|54.9|64.6|
|text-embedding-3-small|62,500|62.3|44.0|
|text-embedding-ada-002|12,500|61.0|31.4|
* MTEB得分为embedding model分类、聚类、配对等八个任务的平均得分。
* MIRACL得分为embedding model在检索任务上的平均得分。  

从以上三个embedding model我们可以看出`text-embedding-3-large`有最好的性能和最贵的价格，当我们搭建的应用需要更好的表现且成本充足的情况下可以使用；`text-embedding-3-small`有着较好的性能跟价格，当我们预算有限时可以选择该模型；而`text-embedding-ada-002`是OpenAI上一代的模型，无论在性能还是价格都不如及前两者，因此不推荐使用。

In [1]:
import os
import openai 
from openai import OpenAI


openai.api_base = "https://openkey.cloud/v1" # 换成代理，一定要加v1
openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxx"  

def openai_embedding(text: str, model: str=None):
    # 获取环境变量 OPENAI_API_KEY
    
    client = OpenAI(
    # This is the default and can be omitted
    base_url = openai.api_base,
    api_key = openai.api_key,
)

    # embedding model：'text-embedding-3-small', 'text-embedding-3-large', 'text-embedding-ada-002'
    if model == None:
        model="text-embedding-3-small"

    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response

response = openai_embedding(text='要生成 embedding 的输入文本，字符串形式。')  
response 

CreateEmbeddingResponse(data=[Embedding(embedding=[0.038827355951070786, 0.013531949371099472, -0.0025024667847901583, -0.016542360186576843, 0.02412303350865841, -0.017386866733431816, 0.042086150497198105, 0.011515072546899319, -0.0282362699508667, -0.006800748407840729, 0.0011779601918533444, 0.01401878148317337, -0.023129498586058617, 0.015827015042304993, -0.0058717927895486355, 0.013790268450975418, -0.01060102041810751, -0.010829533450305462, -0.011475331149995327, 0.023745490238070488, 0.023745490238070488, -0.008946784771978855, -0.007391901686787605, 0.01712854579091072, 0.016820549964904785, -0.022036610171198845, 0.02100333385169506, -0.009970125742256641, 0.04240408167243004, -0.0013934330781921744, -0.08576195687055588, -0.04101313278079033, 0.016522489488124847, -0.0392446406185627, 0.01277686282992363, 0.041648995131254196, -0.008206600323319435, -0.013223953545093536, 0.021698808297514915, 0.016015786677598953, 0.004836032632738352, 0.013005375862121582, -0.00022711593

API返回的数据为`json`格式，除`object`向量类型外还有存放数据的`data`、embedding model 型号`model`以及本次 token 使用情况`usage`等数据，具体如下所示：
```json
{
  "object": "list",
  "data": [
    {
      "object": "embedding",
      "index": 0,
      "embedding": [
        -0.006929283495992422,
        ... (省略)
        -4.547132266452536e-05,
      ],
    }
  ],
  "model": "text-embedding-3-small",
  "usage": {
    "prompt_tokens": 5,
    "total_tokens": 5
  }
}
```
我们可以调用response的object来获取embedding的类型。

In [2]:
print(f'返回的embedding类型为：{response.object}')

返回的embedding类型为：list


embedding存放在data中，我们可以查看embedding的长度及生成的embedding。

In [3]:
print(f'embedding长度为：{len(response.data[0].embedding)}')
print(f'embedding（前10）为：{response.data[0].embedding[:10]}')

embedding长度为：1536
embedding（前10）为：[0.038827355951070786, 0.013531949371099472, -0.0025024667847901583, -0.016542360186576843, 0.02412303350865841, -0.017386866733431816, 0.042086150497198105, 0.011515072546899319, -0.0282362699508667, -0.006800748407840729]


我们也可以查看此次embedding的模型及token使用情况。

In [4]:
print(f'本次embedding model为：{response.model}')
print(f'本次token使用情况为：{response.usage}')

本次embedding model为：text-embedding-3-small
本次token使用情况为：Usage(prompt_tokens=12, total_tokens=12)


## 二、使用文心千帆API
Embedding-V1是基于百度文心大模型技术的文本表示模型，Access token为调用接口的凭证，使用Embedding-V1时应先凭API Key、Secret Key获取Access token，再通过Access token调用接口来embedding text。同时千帆大模型平台还支持bge-large-zh等embedding model。

In [5]:
import requests
import json

def wenxin_embedding(text: str):
    # 获取环境变量 wenxin_api_key、wenxin_secret_key
    api_key = os.environ['QIANFAN_AK']
    secret_key = os.environ['QIANFAN_SK']

    # 使用API Key、Secret Key向https://aip.baidubce.com/oauth/2.0/token 获取Access token
    url = "https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={0}&client_secret={1}".format(api_key, secret_key)
    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    
    # 通过获取的Access token 来embedding text
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/embeddings/embedding-v1?access_token=" + str(response.json().get("access_token"))
    input = []
    input.append(text)
    payload = json.dumps({
        "input": input
    })
    headers = {
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return json.loads(response.text)
# text应为List(string)
text = "要生成 embedding 的输入文本，字符串形式。"
response = wenxin_embedding(text=text)

Embedding-V1每次embedding除了有单独的id外，还有时间戳记录embedding的时间。

In [6]:
print('本次embedding id为：{}'.format(response['id']))
print('本次embedding产生时间戳为：{}'.format(response['created']))

本次embedding id为：as-hvbgfuk29u
本次embedding产生时间戳为：1711435238


同样的我们也可以从response中获取embedding的类型和embedding。

In [7]:
print('返回的embedding类型为:{}'.format(response['object']))
print('embedding长度为：{}'.format(len(response['data'][0]['embedding'])))
print('embedding（前10）为：{}'.format(response['data'][0]['embedding'][:10]))

返回的embedding类型为:embedding_list
embedding长度为：384
embedding（前10）为：[0.060567744076251984, 0.020958080887794495, 0.053234219551086426, 0.02243831567466259, -0.024505289271473885, -0.09820500761270523, 0.04375714063644409, -0.009092536754906178, -0.020122773945331573, 0.015808865427970886]


## 三、使用讯飞星火API

### 尚未开放

## 四、使用智谱API
智谱有封装好的SDK，我们调用即可。

In [9]:
from zhipuai import ZhipuAI
def zhipu_embedding(text: str):

    api_key = os.environ['ZHIPUAI_API_KEY']
    client = ZhipuAI(api_key=api_key)
    response = client.embeddings.create(
        model="embedding-2",
        input=text,
    )
    return response

text = '要生成 embedding 的输入文本，字符串形式。'
response = zhipu_embedding(text=text)

response为`zhipuai.types.embeddings.EmbeddingsResponded`类型，我们可以调用`object`、`data`、`model`、`usage`来查看response的embedding类型、embedding、embedding model及使用情况。

In [10]:
print(f'response类型为：{type(response)}')
print(f'embedding类型为：{response.object}')
print(f'生成embedding的model为：{response.model}')
print(f'生成的embedding长度为：{len(response.data[0].embedding)}')
print(f'embedding（前10）为: {response.data[0].embedding[:10]}')

response类型为：<class 'zhipuai.types.embeddings.EmbeddingsResponded'>
embedding类型为：list
生成embedding的model为：embedding-2
生成的embedding长度为：1024
embedding（前10）为: [0.017892399802803993, 0.0644201710820198, -0.009342825971543789, 0.02707476168870926, 0.004067837726324797, -0.05597858875989914, -0.04223804175853729, -0.03003198653459549, -0.016357755288481712, 0.06777040660381317]


## 五 本地Embedding Model

- Bge-large-zh-v1.5